In [15]:
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import main
import tweepy
import tweet_processing

consumer_key = 'JABuSWGGYTBTQ18FjMvrOJgNQ'
consumer_secret = '3OBRHyALFYS7eClUBEWrjdtupmhee1u1Tt8GYLjngZ6nTkOGIc'

# User access token - Uses user @_tweetsentiment
access_token = '921765888591884288-dDXYnS3luFVZe2qcAzjjssTiNNjf7FM'
access_token_secret = 'tOAUKdwMdUojotllVeSsGFxOjCZAxdEZjhmheKTdVX1Ne'

# OAuth process, using the keys and tokens
try:
    # create OAuthHandler object
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    # set access token and secret
    auth.set_access_token(access_token, access_token_secret)
    # create tweepy API object to fetch tweets
    api = tweepy.API(auth)
except:
    print("Error: Authentication Failed")

sa = main.SentimentAnalysis(False, api, 'bayes', 1, 10)

## Analysis setup and preprocessing

### Importing tweets from Twitter
The first step in the setup process will be to gather the data to be analysed. As we are using Twitter as our source in this case, we will be pulling relevent tweets from Twitter. This is done by using the Tweepy library in Python and results in the below tweets from [@_tweetsentiment](https://twitter.com/_tweetsentiment) ready to proceed:

In [16]:
# Pull the tweets from the users timeline on Twitter using the API
def pull_user_tweets(user, debug):
    user_tweets = sa.api.user_timeline(user, count=sa.tweets_num, tweet_mode='extended')
    return user_tweets
    
def pull_tweets(option, user, search_term):
    tweets_to_process = []
    # Pull the users actual tweets or their timeline tweets
    if option == 'self':
        old_tweets = pull_user_tweets(user, False)
        for tweet in old_tweets:
            if 'retweeted_status' in dir(tweet):
                tweets_to_process.append('RT @' + tweet.retweeted_status.author.screen_name + ': "' + tweet.retweeted_status.full_text + '"')
            else:
                tweets_to_process.append('"' + tweet.full_text + '"')
    # ...Shortened for demonstration purposes
    return tweets_to_process

unprocessed_tweets = pull_tweets('self', '_tweetsentiment', '')

In [17]:
# Print the pulled tweets nicely
print("-----------------------------------")
for tweet in unprocessed_tweets:
    print("- " + tweet)
print("-----------------------------------")

-----------------------------------
- "1 of my tweets were identified as positive and 1 were identified as negative. Analyse your tweets now: https://t.co/TFru1psHj5 via @_tweetsentiment"
- "0 of my tweets were identified as positive and 1 were identified as negative. Analyse your tweets now: https://t.co/TFru1psHj5 via @_tweetsentiment"
- "The website is currently work in progress and nearing completion, watch this space!"
-----------------------------------


### Tweet preprocessing
Now that we have these tweets we will need to preprocess them to remove any 'noise' from them, the noise being any unuseful details from them which don't help us to determine the sentiment.

Preprocessing usually involves a number of steps, for tweet preprocessing these steps include:
* Stripping links
* Stripping special characters
* Stripping emojis and emoticons
* Stripping @mentions
* Trimming repeating words e.g happyyyyyyyyyyy -> happy
* Removing common stop words such as the, a, and etc. 

Below you can see the result when the tweets pulled in the previous step are passed through the preprocessing procedure:

In [18]:
# If there are tweets to process then process them
if unprocessed_tweets != -1:
    # Need to preprocess these tweets now to remove symbols and hashtags etc.
    preprocessed_tweets = tweet_processing.preprocess_tweets(unprocessed_tweets, False)

In [19]:
# Print the processed tweets nicely
print("-----------------------------------")
for tweet in preprocessed_tweets:
    print("- " + tweet)
print("-----------------------------------")

-----------------------------------
- tweets identified positive identified negative analyse tweets via
- tweets identified positive identified negative analyse tweets via
- the website currently work progress nearing completion watch space
-----------------------------------


The procedure of preprocessing is a key part of sentiment analysis especially when it comes to Twitter or any social media platform. There will often be a lot of noise which isn't useful to the sentiment analysis process, so by stripping this out beforehand we can improve the overall efficiency of the procedure. 

Below you can see an example of 3 potential tweets which may be found on Twitter. Now observe the output tweets after preprocessing:

In [20]:
sentences = ["I am happy but i like a lot of cake", "i am lol, well whatever", "i am sad and i hate all of this"]
preprocessed = tweet_processing.preprocess_tweets(sentences, False)

In [21]:
# Print the processed tweets nicely
print("-----------------------------------")
for tweet in sentences:
    print("- " + tweet)
print("-----------------------------------")
for tweet in preprocessed:
    print("- " + tweet)
print("-----------------------------------")

-----------------------------------
- I am happy but i like a lot of cake
- i am lol, well whatever
- i am sad and i hate all of this
-----------------------------------
- happy like lot cake
- lol well whatever
- sad hate
-----------------------------------


### Corpus selection
After pulling the data and preprocessing it, we must choose which training set (corpus) we wish to use in order to train the classifier which we will use in the sentiment analysis. The corpus used for training a sentiment analysis classifier is a key component of the whole process and therefore it is vital that the most optimal corpus is used.

For this specific case there is a choice of three corpus':

1. Tweets - A set of 100,000 tweets, with their sentiment determined manually
2. Tweets - A set of 10,000 tweets taken from the NLTK Twitter Corpus library (5000 positive tweets and 5000 negative tweets) which have had their sentiment determined manually.
3. Film reviews - A set of 2000 film reviews with their sentiment determined manually

Now there is an obvious difference between the first two corpus' and the third which will without doubt make the first two better. In this case, all three are included for comparison purposes and to demonstrate the importance of the training corpus in the overall process. Below is an small sample taken from the first corpus of tweets:

- "is so sad for my APL friend............." - Negative
- "I missed the New Moon trailer..." - Negative
- "omg its already 7:30 :O" - Positive
- ".. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)..." - Negative
- "i think mi bf is cheating on me!!!       T_T" - Negative

## Next step

Now that we have the preprocessed tweets, it is time to classify them. On this site there are currently 2 options to do this:
* Naive Bayesian Classifier
* Support Vector Machine (SVM)

In the next couple of sections I will explain how we can now go on to classify these tweets using a Naive Bayesian or Support vector machine classifier.